# Creating a Custom block, block is just a collection of layers

Necessary steps for defining a block

1. Provision of accepting the input
2. Provision to produce a meaningful output. This is typically encoded in what we will call the forward function. It allows us to invoke a block via net(X) to obtain the desired output.
3. It needs to produce a gradient with regard to its input when invoking backward. Autograd takes care of this part.
4. It needs to store parameters that are inherent to the block.
5. Obviously it also needs to initialize these parameters as needed.

In [10]:
class CustomBlock(nn.Module):

  def __init__(self, **kwargs):
  # Call the constructor of the parent class Module to perform the
  # necessary initialization.
    super(CustomBlock, self).__init__(**kwargs)
    self.hidden = nn.Sequential(nn.Linear(20,256),nn.ReLU()) # Hidden layer
    self.output = nn.Linear(256,10) # Output layer

  # Define the forward computation of the model, that is, how to return the 
  # required model output based on the input x
  def forward(self, x):
    return self.output(self.hidden(x))

In [11]:
net1 = CustomBlock()
x = torch.randn(2,20)
net1(x)

tensor([[-0.2600, -0.2599,  0.1142,  0.0969,  0.0293, -0.2515,  0.0657,  0.0092,
          0.2506,  0.1843],
        [ 0.2142, -0.2933,  0.2687, -0.0835,  0.2717, -0.0418, -0.1790,  0.4063,
          0.1690, -0.1888]], grad_fn=<AddmmBackward>)

#Custom implementation of Sequential layer (i.e. nn.sequential)

In [4]:
import torch
import torch.nn as nn

In [5]:
class MySequential(nn.Sequential):
  def __init__(self, **kwargs):
    super(MySequential, self).__init__(**kwargs)
  def add_module(self, block):
    # Here we assume block has a unique name. We save it in the member 
    # variable _children of the Block class.
    self._modules[block] = block

  def forward(self, x):
    # OrderedDict guarantees that members will be traversed in the order
    # they were added
    for block in self._modules.values():
      x = block(x)
    return x

In [8]:
net = MySequential()
net.add_module(nn.Linear(20,256))
net.add_module(nn.ReLU())
net.add_module(nn.Linear(256,10))
x = torch.randn(2,20)
net(x)

tensor([[ 0.1256,  0.1116,  0.2782, -0.0731, -0.1832,  0.0491,  0.1077, -0.1340,
          0.0596, -0.0135],
        [ 0.2973,  0.1412, -0.0242,  0.0690,  0.0182, -0.0184, -0.2834, -0.0585,
         -0.0611, -0.2731]], grad_fn=<AddmmBackward>)